In [95]:
import pandas as pd
import json


data = pd.read_csv("aggregated_api_request.csv")
data = data.loc[data.project_id == "project-id-3"]

data.timestamp = pd.to_datetime(data.timestamp).apply(lambda x: x.strftime("%H:%M:%S"))
data.to_json("event.json", orient="records", indent=4)

FileNotFoundError: [Errno 2] No such file or directory: 'aggregated_api_request.csv'

In [5]:
from cassandra.cluster import Cluster
from cassandra.query import SimpleStatement
import matplotlib.pyplot as plt
from datetime import datetime
import pandas as pd

def connect_to_cassandra():
    cluster = Cluster(['localhost'])  # Replace with your Cassandra node IP
    session = cluster.connect('spark_streams')  # Replace with your keyspace name
    return session

In [7]:
def query_request_count_by_project(session, project_id, from_date):
    query = f"""
        SELECT * FROM aggregated_api_request
        WHERE project_id = '{project_id}' AND timestamp >= '{from_date}';
    """
    statement = SimpleStatement(query)
    result = session.execute(statement)

    return result

In [15]:
session = connect_to_cassandra()
project_id = 'project-id-1'  # Replace with the desired project ID# Replace with the desired month bucket
start = "2023-11-01 00:00:00.000"

result = query_request_count_by_project(session, project_id, start)
df = pd.DataFrame(result)

df_resampled =  df.set_index("timestamp").resample("10Min").sum().fillna(0).reset_index()
project_id = "project-id-1"

df_resampled['project_id'] = [project_id if project == 0 else project for project in df_resampled['project_id']]
df_resampled

,timestamp,project_id,avg_duration,count,event_type,fiftieth_per_duration,ninetieth_per_duration,ninety_fifth_per_duration
0,2023-11-30 07:10:00,project-id-1,7222.222168,9,PREDICT_FAILED,6000.0,14000.0,14000.0
1,2023-11-30 07:20:00,project-id-1,7902.173828,92,PREDICT_FAILED,8000.0,14000.0,14000.0
2,2023-11-30 07:30:00,project-id-1,7721.311523,183,PREDICT_COMPLETED,7000.0,14000.0,15000.0
3,2023-11-30 07:40:00,project-id-1,8430.051758,193,PREDICT_COMPLETED,8000.0,15000.0,15000.0
4,2023-11-30 07:50:00,project-id-1,7560.000000,25,PREDICT_COMPLETED,8000.0,15000.0,15000.0
...,...,...,...,...,...,...,...,...
143,2023-12-01 07:00:00,project-id-1,7111.111328,18,PREDICT_FAILED,6000.0,12000.0,15000.0
144,2023-12-01 07:10:00,project-id-1,7767.856934,56,PREDICT_COMPLETED,8000.0,15000.0,15000.0
145,2023-12-01 07:20:00,project-id-1,0.000000,0,0,0.0,0.0,0.0
146,2023-12-01 07:30:00,project-id-1,8285.713867,84,PREDICT_COMPLETED,9000.0,14000.0,15000.0


In [69]:
import pytz



def process_dataframe_for_last(df, time_range='6_hours'):
    
    end_time = pd.to_datetime('now')
    timedelta = 10
    # Determine the start time based on the specified time range
    if time_range == '6_hours':
        end_time = end_time.replace(second=0, microsecond=0, minute=0, hour=end_time.hour+1)
        start_time = (end_time - pd.Timedelta(hours=6))
    elif time_range == '1_day':
        end_time = pd.to_datetime(end_time.date()) + pd.DateOffset(days=1)
        start_time = (end_time - pd.Timedelta(days=1)).date()
        timedelta = 60
    elif time_range == '7_day':
        end_time = pd.to_datetime(end_time.date()) + pd.DateOffset(days=1) - pd.Timedelta(seconds=1)
        timedelta = 60 * 24
        start_time = pd.to_datetime((end_time - pd.Timedelta(days=8)).date())
    else:
        raise ValueError("Invalid time_range. Supported values are '6_hours' and 'one_day'.")

    desired_timezone = 'Asia/Bangkok'
    df['timestamp'] = pd.to_datetime('now').tz_localize(pytz.timezone(desired_timezone)).strftime('%Y-%m-%d %H:%M:%S')
    # Filter data for the specified time range
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df_filtered = df[df['timestamp'] >= start_time]

     # Convert 'timestamp' to DateTime object

    if time_range == '6_hours':
        df_filtered['formatted_timestamp'] = df_filtered['timestamp'].dt.round('10min')
    elif time_range == '1_day':
        df_filtered['formatted_timestamp'] = df_filtered['timestamp'].dt.floor('H')
    elif time_range == '7_day':
        df_filtered['formatted_timestamp'] = df_filtered['timestamp'].dt.floor('D')

    # Group by formatted timestamp and calculate count
    grouped_df = df_filtered.groupby(['formatted_timestamp'])['count'].sum().reset_index()

    chart_data = {
        'timestamp': [],
        'count': []
    }

    def formatTimestime(timestamp): 
        if time_range == '6_hours':
            return timestamp.strftime('%H:%M')
        elif time_range == '1_day':
            return timestamp.strftime('%H')
        else:
            return timestamp.strftime('%Y-%m-%d')

    current_time = start_time
    while current_time <= end_time:
        count_value = grouped_df.loc[grouped_df['formatted_timestamp'] == current_time, 'count'].values
        chart_data['count'].append(count_value[0] if len(count_value) > 0 else 0)
        chart_data['timestamp'].append(formatTimestime(current_time))
        current_time += pd.Timedelta(minutes=timedelta)     
    
     
    return df_filtered


session = connect_to_cassandra()
project_id = 'project-id-1'  # Replace with the desired project ID# Replace with the desired month bucket
start = "2023-11-01 00:00:00.000"

result = query_request_count_by_project(session, project_id, start)
df = pd.DataFrame(result)
res = process_dataframe_for_last(df, '6_hours')
res

,project_id,timestamp,avg_duration,count,event_type,fiftieth_per_duration,ninetieth_per_duration,ninety_fifth_per_duration,formatted_timestamp
0,project-id-1,2023-12-01 15:45:49,7222.222168,9,PREDICT_FAILED,6000.0,14000.0,14000.0,2023-12-01 15:50:00
1,project-id-1,2023-12-01 15:45:49,7902.173828,92,PREDICT_FAILED,8000.0,14000.0,14000.0,2023-12-01 15:50:00
2,project-id-1,2023-12-01 15:45:49,7721.311523,183,PREDICT_COMPLETED,7000.0,14000.0,15000.0,2023-12-01 15:50:00
3,project-id-1,2023-12-01 15:45:49,8430.051758,193,PREDICT_COMPLETED,8000.0,15000.0,15000.0,2023-12-01 15:50:00
4,project-id-1,2023-12-01 15:45:49,7560.000000,25,PREDICT_COMPLETED,8000.0,15000.0,15000.0,2023-12-01 15:50:00
5,project-id-1,2023-12-01 15:45:49,7111.111328,18,PREDICT_FAILED,6000.0,12000.0,15000.0,2023-12-01 15:50:00
6,project-id-1,2023-12-01 15:45:49,7767.856934,56,PREDICT_COMPLETED,8000.0,15000.0,15000.0,2023-12-01 15:50:00
7,project-id-1,2023-12-01 15:45:49,8285.713867,84,PREDICT_COMPLETED,9000.0,14000.0,15000.0,2023-12-01 15:50:00
8,project-id-1,2023-12-01 15:45:49,9000.000000,1,PREDICT_FAILED,9000.0,9000.0,9000.0,2023-12-01 15:50:00
9,project-id-1,2023-12-01 15:45:49,7185.185059,27,PREDICT_FAILED,7000.0,15000.0,15000.0,2023-12-01 15:50:00
